In [10]:
from pathlib import Path
import json
import h5py
import os 
HERE = Path().resolve()          # /path/to/your/script
json_path = HERE / 'config.json'                 # /path/to/your/script/config.json
with json_path.open('r', encoding='utf-8') as f:
    config = json.load(f)
print(config['grid']['Nx'])          # 512
print(config['medium']['vinyl'])     # {'sound_speed': 2390, ...}
save_path = config['save_path']
signal_path = os.path.join(save_path, 'liquid_only.mat')
with h5py.File(signal_path, 'r') as f:
    def print_tree(name, obj):
        if isinstance(obj, h5py.Dataset):
            print(f'Dataset: {name}  shape={obj.shape} dtype={obj.dtype}')
        else:
            print(f'Group:   {name}')
    f.visititems(print_tree)

512
{'sound_speed': 2390, 'density': 1400, 'alpha_coeff': 0.448, 'alpha_power': 2, 'BonA': 6}
Group:   #refs#
Dataset: #refs#/a  shape=(2,) dtype=uint64
Dataset: #refs#/b  shape=(1, 584) dtype=uint8
Dataset: #refs#/c  shape=(1, 1) dtype=float64
Dataset: #refs#/d  shape=(1, 1) dtype=float64
Dataset: #refs#/e  shape=(1, 1) dtype=float64
Dataset: #refs#/f  shape=(1, 1) dtype=float64
Dataset: #refs#/g  shape=(1, 1) dtype=float64
Dataset: #refs#/h  shape=(1, 1) dtype=float64
Dataset: #refs#/i  shape=(1, 512) dtype=float64
Dataset: #refs#/j  shape=(1, 512) dtype=float64
Dataset: #refs#/k  shape=(1, 128) dtype=float64
Dataset: #refs#/l  shape=(128, 512, 512) dtype=float64
Dataset: #refs#/m  shape=(1, 1) dtype=float64
Dataset: #refs#/n  shape=(1, 1) dtype=float64
Dataset: #refs#/o  shape=(1, 1) dtype=float64
Dataset: #refs#/p  shape=(1, 1) dtype=float64
Dataset: #refs#/q  shape=(1, 1) dtype=float64
Dataset: #refs#/r  shape=(1, 1) dtype=float32
Dataset: #refs#/s  shape=(1, 1) dtype=float32
Data